In [1]:
import numpy as np
import pandas as pd
import tqdm
from scipy import sparse
import scipy
from sklearn.metrics.pairwise import cosine_similarity
from multiprocesspandas import applyparallel
from scipy.sparse import csr_matrix

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Prepare 

Создается новый датасет с заполненной new_id
- если у юзера есть user_id -> user_id = new_id
- если у юзера нет user_id -> user_id = cookie_id
- у юзеров, где пропущены user_id заполняются пропуски

In [ ]:
# df = pd.read_parquet('train_mfti.parquet', engine='pyarrow')

# users_cookies = df[df['user_id'].notna()].groupby(['cookie_id'], as_index=False)['user_id'].agg(['unique'])
# users_cookies_list = users_cookies.index

In [ ]:
# df.drop_duplicates(inplace=True)

In [ ]:
# def set_id_unknown(cookie):
#     if cookie in users_cookies_list:
#         return users_cookies.loc[cookie][0][0]
#     else:
#         return cookie
    
# arr_new_id = list()
# for i in tqdm.tqdm(df['cookie_id']):
#     arr_new_id.append(set_id_unknown(i))
    
# df.insert(6, "new_id", arr_new_id, True)
# print('Нет пропусков id',df[df['new_id'].isna()]['new_id'].sum() == 0)

In [ ]:
# df.to_csv('with_new_ids_drop_dub.csv', index=False)

---

# Чтение датасетов
- заменяем cookie_id в тестовом датасете на new_id

In [4]:
df = pd.read_csv('with_new_ids_drop_dub.csv', engine='c')
df.drop(['user_id', 'event_date'], axis=1, inplace=True)
df

,event_timestamp,vacancy_id_,cookie_id,event_type,new_id
0,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,show_vacancy,951f53de61764ea0b51317200a0dbbfc
1,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,show_vacancy,f5a2326a17484330aa8cb4019f1b1960
2,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,show_vacancy,f5a2326a17484330aa8cb4019f1b1960
3,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,show_vacancy,f5a2326a17484330aa8cb4019f1b1960
4,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,show_vacancy,f5a2326a17484330aa8cb4019f1b1960
...,...,...,...,...,...
12246984,1660246479,162851,0d3c58c6864546c689fa0997de484b30,preview_click_response,18124998fd644e80b667a1d84bf63846
12246985,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,preview_click_response,5c71c1f1b7cf492ba05871c400a05250
12246986,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,preview_click_response,33c968125459420e89ffbc8cffece317
12246987,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,preview_click_response,33c968125459420e89ffbc8cffece317


In [5]:
uniq_users_id = df['new_id'].unique()
uniq_vacancy = df['vacancy_id_'].unique()
users_cookies = df.groupby(['cookie_id'], as_index=False)['new_id'].agg(['unique'])

In [6]:
interactions = df.groupby(['new_id', 'vacancy_id_', 'event_type'], as_index=False) \
.agg({'event_timestamp': 'count'}) \
.sort_values(['event_timestamp'], ascending=False)

In [7]:
test_df = pd.read_parquet('test_public_mfti.parquet', engine='pyarrow')
test_df

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [8]:
def get_new_id_by_cookie(cookie):
    return users_cookies.loc[cookie][0][0]

In [9]:
test_df['cookie_id'] = test_df['cookie_id'].apply(lambda x: get_new_id_by_cookie(x))
test_df = test_df.rename(columns={'cookie_id': 'new_id'})

In [10]:
test_df

,new_id,vacancy_id_
0,477e719bfbcc4071a8acc1b357492b00,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,753a2a268ae84ab49475e62736c02860,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,d01c76b282364fbb8195f326de3e893c,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,6adb4495090b454395d5aa2f1a5ff9d9,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,b107696e3ff54e8a83182828bb845bc6,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,da74810d481449278a75a091178f9831,"[207851, 254989, 213344, 214180, 222146]"
768,4da39c1e048d49cabcd1229fdcc6413f,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,5207e9855ec54e02aa8cfa48ede76e1b,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,7f2067ab30994440b13a0ea37a5e8001,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [12]:
# default vacancies to give
top_vacancies = df.pivot_table(index='vacancy_id_', columns='event_type', values='event_timestamp', aggfunc='count', fill_value=0)
top_vacancies['ctr'] = top_vacancies.apply(lambda x: (x['preview_click_vacancy'] + x['click_response'] + x['click_contacts'] + x['preview_click_response']+ x['click_favorite'] + x['preview_click_favorite'] + x['preview_click_contacts'] + x['click_phone'] + x['preview_click_phone'])/x['show_vacancy'] if x['show_vacancy'] != 0 else 0, axis=1)
top_vacancies = list(top_vacancies.sort_values(['ctr'], ascending=False).index[:10])

# Matrix factorization

- Выбираем как считать заинтересованность юзера вакансией

In [ ]:
def help_func(x, events):
    if ((len(events)>0) and ('show_vacancy' in events) and (x['show_vacancy']>0)):
        return round((sum([x[i] for i in events]) - x['show_vacancy']) / x['show_vacancy'], 1)
    elif (('show_vacancy' not in events) or (x['show_vacancy'] == 0)):
        return sum([x[i] for i in events])
    else:
        return 0
    
def help_func_sum(x, events):
    return len([x[i] for i in events if x[i]])

def get_vacancies_ctr():
    unique_user_events = interactions['event_type'].unique().tolist()

    vacancy_table = interactions.pivot_table(index=['new_id', 'vacancy_id_'], columns='event_type', values='event_timestamp', fill_value=0).reset_index()
    vacancy_table['ctr'] = vacancy_table.apply_parallel(help_func, events=unique_user_events, num_processes=6)
    
    return vacancy_table[['new_id', 'vacancy_id_', 'ctr']]


def get_vacancies_sum_clicks():
    interactions = df.query('event_type != "preview_click_vacancy" and event_type != "show_vacancy"') \
    .groupby(['new_id', 'vacancy_id_', 'event_type'], as_index=False) \
    .agg({'event_timestamp': 'count'})
    
    unique_user_events = interactions['event_type'].unique().tolist()
    vacancy_table = interactions.pivot_table(index=['new_id', 'vacancy_id_'], columns='event_type', values='event_timestamp', fill_value=0).reset_index()
    vacancy_table['ctr'] = vacancy_table.apply_parallel(help_func_sum, events=unique_user_events, num_processes=6)
    
    return vacancy_table[['new_id', 'vacancy_id_', 'ctr']]

In [ ]:
# Новые датафрейм, где для каждой вакансии юзера, есть степень интереса
# df_new = get_vacancies_sum_clicks()
# df_new.to_csv('file_with_get_vacancies_sum_clicks.csv', index=False)

In [13]:
df_new = pd.read_csv('file_with_user_vacancy_ctr.csv')
df_new['ctr'] = df_new['ctr'].astype('float32')

In [14]:
df_new

,new_id,vacancy_id_,ctr
0,000089d26cdd49d68839c68bc10f2cf2,177380,1.0
1,000089d26cdd49d68839c68bc10f2cf2,181353,1.0
2,000089d26cdd49d68839c68bc10f2cf2,195890,1.0
3,000089d26cdd49d68839c68bc10f2cf2,205562,1.0
4,000089d26cdd49d68839c68bc10f2cf2,211064,1.0
...,...,...,...
4661345,ffffdb17f45b4032b386d691d52e6c00,107125,2.0
4661346,ffffdb17f45b4032b386d691d52e6c00,107893,2.0
4661347,ffffdb17f45b4032b386d691d52e6c00,150843,2.0
4661348,ffffdb17f45b4032b386d691d52e6c00,160164,2.0


In [15]:
def proc_col(col, train_col=None):
# Encodes a pandas column with continous ids. 
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

In [16]:
mapping = []
def encode_data(df, train=None):
    mask = df['new_id'] > ''
    # Encodes rating data with continous user and movie ids. 
    # If train is provided, encodes df with the same encoding as train.
    df = df.copy()
    for col_name in ["new_id", "vacancy_id_"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        name2idx, col, _ = proc_col(df[col_name], train_col)  
        mapping.append(name2idx)
        df[col_name] = col
        df = df[df[col_name] >= 0]
        mask = mask & (df[col_name] > 0)
    return df, mask

In [17]:
np.random.seed(3)
msk = np.random.rand(len(df_new)) < 0.9
train = df_new[msk].copy()
val = df_new[~msk].copy()

In [18]:
# # encoding the train and validation data
df_train,_ = encode_data(train)
df_val,_  = encode_data(val, train)

---

# Train

In [ ]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u*v).sum(1)  
    

num_users = len(df_train.new_id.unique())
num_items = len(df_train.vacancy_id_.unique())
print(num_users, num_items) 

model = MF(num_users, num_items, emb_size=100)

def train_epocs(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    for i in tqdm.tqdm(range(epochs)):
        users = torch.LongTensor(df_train.new_id.values)
        items = torch.LongTensor(df_train.vacancy_id_.values)
        ratings = torch.FloatTensor(df_train.ctr.values)
        if unsqueeze:
            ratings = ratings.unsqueeze(1)
        y_hat = model(users, items)
        loss = F.mse_loss(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(loss.item()) 
    test_loss(model, unsqueeze)
    
def test_loss(model, unsqueeze=False):
    model.eval()
    users = torch.LongTensor(df_val.new_id.values)
    items = torch.LongTensor(df_val.vacancy_id_.values)
    ratings = torch.FloatTensor(df_val.ctr.values)
    if unsqueeze:
        ratings = ratings.unsqueeze(1)
    y_hat = model(users, items)
    loss = F.mse_loss(y_hat, ratings)
    # print("test loss %.3f " % loss.item())

---

In [ ]:
interactions = df.groupby(['new_id', 'vacancy_id_', 'event_type'], as_index=False) \
.agg({'event_timestamp': 'count'}) \
.sort_values(['event_timestamp'], ascending=False)

uniq_vacancy = df['vacancy_id_'].unique()

new_test_df = pd.read_csv('new_test for model.csv')

# Получить использованные вакансии
def get_not_used_vacancies(new_id):
    used = interactions[interactions['new_id'] == new_id]['vacancy_id_'].unique()
    useful_vacs = list()
    for i in uniq_vacancy:
        if i not in used:
            useful_vacs.append(i)
    return useful_vacs


def get_recommendation(new_test_df):
    df_test, mask = encode_data(new_test_df, train)
    no_pred = new_test_df[~mask].copy()
    new_test_df = new_test_df[mask].copy()

    users = torch.LongTensor(df_test.new_id.values) # .cuda()
    items = torch.LongTensor(df_test.vacancy_id_.values) #.cuda()
    y_hat = model(users, items)
    new_test_df['ctr'] = y_hat.detach().numpy().tolist()
    return new_test_df.sort_values(by=['new_id','ctr'], ascending=False), no_pred

def get_precision(test_df):
    res_df, no_pred = get_recommendation(new_test_df)

    res_df = res_df[res_df.groupby('new_id')['ctr'].rank(method='first', ascending=True) <= 5]
    res_df = res_df.groupby(['new_id'])['vacancy_id_'].agg(['unique'])
    if len(test_df) != len(res_df):
        diff = np.setdiff1d(test_df['new_id'], res_df.index)
        tmp_diff = pd.DataFrame({'unique': [top_vacancies[:5]]*len(diff)}, index=diff)
        res_df = pd.concat([res_df, tmp_diff])
    res_df = res_df.sort_index(ascending=False)

    test_df = test_df.sort_values(by=['new_id'], ascending=False)
    test_df['pred'] = res_df['unique'].values
    test_df['true_positive'] = test_df.apply(lambda x: len(set(x['vacancy_id_']) & set(x['pred'])), axis=1)
    test_df['f5'] = test_df.apply(lambda x: len(set(x['vacancy_id_']) & set(x['pred'])) / 5, axis=1)
    print('f5: ', test_df['f5'].mean())

In [ ]:
for epoch in [50, 100, 200]:
    for lr in [0.01, 0.02]:
        model = MF(num_users, num_items, emb_size=100)
        train_epocs(model, epochs=epoch, lr=lr)
        print("Epoch: ", epoch)
        print("Learning rate: ", lr)
        get_precision(test_df)

In [ ]:
# test_df[test_df['true_positive'] != 0]

In [ ]:
# res = get_recommendation_for_user('477e719bfbcc4071a8acc1b357492b00')
# res
# print(test_df['vacancy_id_'][0])

# AlternatingLeastSquares

In [19]:
from implicit.bpr import BayesianPersonalizedRanking
from implicit.als import AlternatingLeastSquares

In [41]:
matrix_for_fact = sparse.csr_matrix((df_train['ctr'], (df_train['new_id'], df_train['vacancy_id_'])))

In [47]:
def train_fact(factors, regularization, alpha=1, iters=20, rs=30):
    print('RS: ', rs)
    print('Iteration: ', iter)
    
    model = AlternatingLeastSquares(
                                    factors=factors,
                                    regularization=regularization,
                                    alpha=alpha,
                                    calculate_training_loss=True, 
                                    iterations=iters,
                                    random_state=rs
    )
    
    model.fit(matrix_for_fact)
    predict = []
    for i in test_df.itertuples():
        index = i[3]
        if index == -1:
            recomm = top_vacancies[:5]
        else:
            recomm = model.recommend(index, matrix_for_fact[index], N=5)[0]
        res = [i for i,k in mapping[1].items() if k in recomm]
        f = len(set(i[2]) & set(res)) / 5
        predict.append(f)
        
    print(f'f5: {np.mean(predict)}')

In [32]:
test_df['test_id'] = test_df.apply(lambda x: mapping[0].get(x['new_id'], -1), axis=1)

In [43]:
test_df

,new_id,vacancy_id_,test_id
0,477e719bfbcc4071a8acc1b357492b00,"[222177, 222173, 222163, 238874, 238878, 22812...",90150
1,753a2a268ae84ab49475e62736c02860,"[102794, 137587, 257319, 237756, 240744, 11348...",147477
2,d01c76b282364fbb8195f326de3e893c,"[254292, 164602, 116438, 228634, 218819, 24065...",261809
3,6adb4495090b454395d5aa2f1a5ff9d9,"[246685, 138123, 115420, 210628, 212325, 235196]",134531
4,b107696e3ff54e8a83182828bb845bc6,"[219070, 251469, 166899, 212703, 214561]",222798
...,...,...,...
767,da74810d481449278a75a091178f9831,"[207851, 254989, 213344, 214180, 222146]",274679
768,4da39c1e048d49cabcd1229fdcc6413f,"[115352, 230546, 225527, 120188, 109360, 23212...",98000
769,5207e9855ec54e02aa8cfa48ede76e1b,"[240362, 114852, 253946, 251081, 127546, 244688]",103480
770,7f2067ab30994440b13a0ea37a5e8001,"[231194, 236363, 220747, 244688, 100094, 24052...",159965


In [48]:
for factors in [100]:
    print('=====================================================')
    for regularization in [0.05]:
            print('factors', factors,
                  'regularization', regularization)
            train_fact(factors, regularization)

factors 10 regularization 0.05
RS:  30
Iteration:  <built-in function iter>


  0%|          | 0/20 [00:00<?, ?it/s]

f5: 0.03471502590673576


# Cosine similarity

In [ ]:
# Создается массив векторов
def create_massive(original_df, filename):
    uniq_users_id = original_df['new_id'].unique()
    uniq_vacancy = original_df['vacancy_id_'].unique()
    size = len(uniq_vacancy)
    
    interactions = original_df.query('event_type != "preview_click_vacancy" and event_type != "show_vacancy"') \
    .groupby(['new_id', 'vacancy_id_', 'event_type'], as_index=False) \
    .agg({'event_timestamp': 'count'})

    massive = csr_matrix((len(uniq_users_id), size), dtype = np.int8)
    for k1, i in enumerate(tqdm.tqdm(uniq_users_id)):
        user_vacancies = set(interactions[interactions['new_id'] == i]['vacancy_id_'])
        for k2, w in enumerate(uniq_vacancy):
            if w in user_vacancies: massive[k1,k2] = 1 # юзер - вакансия

    scipy.sparse.save_npz('{}.npz'.format(filename), massive)
    return massive

In [ ]:
# Get vectorized users
sparse_vectors = create_massive(df)

# Рассчитывается косинус схожести для sparse массива, в итоге получаем похожих юзеров в виде координат
similarity = cosine_similarity(sparse_vectors, dense_output=False)
scipy.sparse.save_npz('similarity_results.npz', changed_similarity)

In [ ]:
massive = scipy.sparse.load_npz('sparse_massive.npz')

In [ ]:
changed_similarity = scipy.sparse.load_npz('changed_similarity_results.npz')
A = pd.DataFrame.sparse.from_spmatrix(changed_similarity)
display(A) # на пересечении координат косинус похожести

In [ ]:
# f5 = 0.0189 - only cosine similarity

# куки и соответствующие id юзеров
users_cookies = df.groupby(['cookie_id'], as_index=False)['new_id'].agg(['unique'])

# тут уже все действия юзеров
interactions = df.groupby(['new_id', 'vacancy_id_', 'event_type'], as_index=False) \
.agg({'event_timestamp': 'count'}) \
.sort_values(['event_timestamp'], ascending=False)



# Получить new_id/user_id по куки
def get_new_id_by_cookie(cookie):
    return users_cookies.loc[cookie][0][0]

# Лист всех уникальных юзеров по-порядку, в котором составлялась матрица
user_unique_columns = df['new_id'].unique()
# Получить индекс юзера в списке
def get_user_index(user_id):
    return np.where(user_unique_columns == user_id)[0][0]

# Получение вакансий для рекомендации
def get_similar_users_ordered(user_index):
    similar_users_index_list = list()
    for i, val in enumerate(A[user_index]):
        if (val > 0.3) and (i != user_index):
            similar_users_index_list.append((i, val))
    similar_users_index_list = sorted(similar_users_index_list, key = lambda x: x[1], reverse=True)
    return similar_users_index_list

# Ранжирование товаров юзера
def get_vacancies_ordered_by_user(new_id):
    user_interaction = interactions[interactions['new_id'] == new_id]
    unique_user_events = user_interaction['event_type'].unique()

    vacancy_table = user_interaction.pivot_table(index='vacancy_id_', columns='event_type', values='event_timestamp', fill_value=0).reset_index()
    vacancy_table['ctr'] = vacancy_table.apply(lambda x: (sum([x[i] for i in unique_user_events])), axis=1)
    vac_dict = list([[v['vacancy_id_'] , v['ctr']] for i, v in vacancy_table.iterrows()])
    return sorted(vac_dict, key = lambda x: x[1], reverse=True)

# Получение отранжированных вакансий по всем юзерам
def get_ordered_vacancies(similar_users_list):
    recommend_vac = list()
    for i in similar_users_list[:10]: # пока беру первых 5 похожих юзеров
        user = user_unique_columns[i[0]]
        vacancies = get_vacancies_ordered_by_user(user)
        for v in vacancies:
            recommend_vac.append(v[0])
    recommend_vac = list(dict.fromkeys(recommend_vac))
    return recommend_vac


# Получить использованные вакансии
def get_used_vacancies(new_id):
    return interactions[interactions['new_id'] == new_id]['vacancy_id_'].unique()
    
    
# Получение рекомендации + в рекомендации учитваем топ вакансий
def get_user_recommendation(user_id):
    recommend_result = []
    index = get_user_index(user_id)
    similar_index_list = get_similar_users_ordered(index)
    
    all_recommended_vac = get_ordered_vacancies(similar_index_list)
    users_used_vacs = get_used_vacancies(user_id)
    
    if len(all_recommended_vac) > 0:
        for i in all_recommended_vac:
            if i not in users_used_vacs:
                recommend_result.append(i)
    else:
        return top_vacancies
    recommend_result = top_vacancies + recommend_result
    return recommend_result

In [ ]:
# Для тестового датасета добавляем колонку pot_vacancies, где будут указаны потенциальные рекомендации на основе косинуса схожести
def make_prediciton(idx):
    pot_vacs = get_user_recommendation(idx)
    return pot_vacs

test_df['pot_vacancies'] = test_df.apply(lambda x: make_prediciton(x['new_id']), axis=1)

In [ ]:
# Преобразовываем датафейм для использования модели, которая сранжирует потенциальные вакансии
# test_df['pot_vacancies'] - потенциальные
users = []
vacancies = []

for i in test_df.itertuples():
    users = users + ([i[1]] * len(i[3]))
    vacancies = vacancies + i[3]
    
data = {'new_id': users, 'vacancy_id_': vacancies}
new_test = pd.DataFrame(data)

In [ ]:
new_test.to_csv('new_test for model_pot 10.csv', index=False)

In [ ]:
new_test_df = pd.read_csv('new_test for model.csv')

In [ ]:
new_test_df